In [1]:
#!pip install numpy --upgrade
import numpy as np
import pandas as pd
import re

from IPython.display import clear_output

#!pip install "gensim"
from gensim.models import Word2Vec   # Word2Vec model
from gensim.models import KeyedVectors

from sklearn.neural_network import MLPClassifier

import time

In [2]:
tr_wv = KeyedVectors.load('/home/x-mbemus/Desktop/train-text-50.vecs').wv   # Load in word vectors.

In [3]:
#!pip install pickle-mixin
import pickle as pkl

In [4]:
df = pd.read_csv("/home/x-mbemus/Desktop/text_train_data.csv")   # Import training data.
df.head()

,Unnamed: 0,id,title,author,pub_date,genres,summary,link,is_novel,sum_len,file_name
0,2,6921,Carmilla,Sheridan Le Fanu,1872,fiction,The story is presented by Le Fanu as part of ...,https://www.gutenberg.org/files/10007/10007-0.txt,False,934,10007_10007-0.txt
1,3,7923,Dracula,Bram Stoker,1897,horror,"The novel is told in epistolary format, as a ...",https://www.gutenberg.org/files/345/345-0.txt,True,986,345_345-0.txt
2,4,7923,Dracula,Bram Stoker,1897,horror,"The novel is told in epistolary format, as a ...",https://www.gutenberg.org/files/45839/45839-0.txt,True,986,45839_45839-0.txt
3,5,8237,Don Quixote,Miguel de Cervantes,1605,novel,"The First Sally Alonso Quijano, the protagoni...",https://www.gutenberg.org/files/996/996-0.txt,True,1592,996_996-0.txt
4,9,13535,Heart of Darkness,Joseph Conrad,1899,novel,'Heart of Darkness' opens in first person nar...,https://www.gutenberg.org/files/219/219-0.txt,True,3127,219_219-0.txt


In [5]:
# Assign classifier.
base_nn = MLPClassifier(random_state=42, hidden_layer_sizes=(100, 100, 100), max_iter=1000)

In [6]:
tr_wv['Dracula']   # Check vectors loaded properly.

array([-0.38713482,  0.35845155, -0.4892324 ,  0.10323673,  0.21779941,
        0.18955876,  0.6996074 ,  0.22049618, -0.26312035,  0.28026226,
        0.41193014, -0.18090388, -0.24702743,  0.483984  ,  0.01322602,
        0.23626558,  0.3272087 , -0.00264295, -0.24421367, -0.5000111 ,
        0.2437251 ,  0.67580444, -0.16511494,  0.21222222,  0.33947915,
        0.04529685, -0.01860966,  0.05741313, -0.47610345,  0.07334362,
        0.04463233, -0.32907084, -0.11431769, -0.44045985, -0.22120006,
        0.1330758 ,  0.21142244,  0.14917983,  0.34179702,  0.4040718 ,
        0.17557351, -0.05840934, -0.0798966 ,  0.17631772,  0.3998845 ,
       -0.08932945,  0.17433514,  0.02371125, -0.05882114,  0.4896915 ],
      dtype=float32)

In [8]:
# Single observation to test on.
f = open("/home/x-mbemus/Desktop/text_files/" + df.at[0, "file_name"])
text = ' '.join(f.readlines())   # Read in file.
# Preprocess as before.
text = re.sub(r"\\n|\\t", " ", text)
text = re.sub(r"([^\w\s'])", r" \1 ", text)
text = re.sub(r"\s+", " ", text)
words = text.split(" ")   # Split tokens by space.

text_vec = np.array([0] * 50)   # Create empty array.
ttl_wrd = 0   # Count words.

for j in words:   # For each word.
    try:
        text_vec = text_vec + tr_wv[j]   # Vectorize word and add to empty array.
        ttl_wrd += 1   # Increment word count.
    except:
        pass

x_0 = text_vec/ttl_wrd   # Create mean vector.
x_0 = x_0.tolist()   # Easier to use as list.

text = df.at[0, "summary"]   # Read in summary.
# Preprocess summary.
text = re.sub(r"\\n|\\t", " ", text)
text = re.sub(r"\\'", " ", text)
text = re.sub(r"([^\w\s'])", r" \1 ", text)
text = re.sub(r"\s+", " ", text)
words = text.split(" ")   # Split tokens by space.
# Create first observation input.
inp = [0] * 150   # Empty tri-gram.
inp = x_0 + inp + [words[0]]   # Mean vector, tri-gram, target output.
full_inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))   # Convert to data frame.

for j in range(1, len(words)):   # For each word in summary.
    try:
        inp = tr_wv[j-1].tolist()   # Try to convert last word to vector.
    except:
        inp = [0] * 50   # If not in vectors, use empty vector instead.
        
    # Mean vector, previous word, two words before that, target word. 
    inp = x_0 + inp + list(full_inp.iloc[j-1, 50:150]) + [words[j]]  
    inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))   # Convert observation to DF.
    full_inp = pd.concat([full_inp, inp]).reset_index(drop=True)   # Add to df.

full_inp.replace(["", np.nan], 0)   # Replace all empty values with NA's.

# Data-type dictionary.
d_dict = {}
for j in range(0, 200):   # First 200 should be floats.
    d_dict[j] = float
d_dict[200] = str   # Last should be string.

full_inp = full_inp.astype(d_dict)   # Convert full df to dictionary.

base_nn.fit(full_inp.iloc[:, 0:199], full_inp.loc[:, 200])   # Train model.

MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
              random_state=42)

In [8]:
print(tr_wv[""])   # Test embedding for empty.
#words[-1]

[ 1.1474203   1.5263748  -1.7495593   3.5340645   1.0957179   2.441623
  2.2997258  -3.2771723  -0.03800014 -2.2002282   2.6911736   4.5104384
 -2.5005765   3.8492079   1.3173671   1.2771536   1.0527943   1.4081445
 -1.292123    1.3697222   1.0598265   0.6523853  -0.10165195  3.3674347
 -3.9011934  -2.0526927   1.0369684  -2.7872827  -1.4726132   4.078271
  2.3434234  -0.31009197  3.7382123  -6.4433727   2.3364267  -1.0388134
  0.00880193  2.809391   -0.07413822  2.4750824   1.2271886   1.594003
 -0.7169712  -1.7197332   4.200405   -1.7136725   3.305261   -2.1560016
 -1.0708046   0.83876747]


In [9]:
for i in df.index:   # Read in texts to data frame.
    try:
        # Open text file.
        f = open("/home/x-mbemus/Desktop/text_files/" + df.at[i, "file_name"], "r", errors="ignore")
        text = ' '.join(f.readlines())   # Read in text.
        # Preprocess as always.
        text = re.sub(r"\\n|\\t", " ", text)
        text = re.sub(r"([^\w\s'])", r" \1 ", text)
        text = re.sub(r"\s+", " ", text)
        
        # Add to data frame.
        df.at[i, "full_text"] = text
    except:
        pass

In [10]:
# Check effect of dropping, then drop.
print(len(df.index))
print(len(df.loc[:]['full_text'].dropna().index))

df.dropna(inplace=True)

666
553


In [11]:
for i in df.index:   # Preprocess summaries.
    text = df.at[i, "summary"]   # Read in summaries.
    # Preprocess as before.
    text = re.sub(r"\\n|\\t", " ", text)
    text = re.sub(r"\\'", " ", text)
    text = re.sub(r"([^\w\s'])", r" \1 ", text)
    text = re.sub(r"\s+", " ", text)
    df.at[i, "summary"] = text   # Add summary to df.

In [12]:
df = df.reset_index(drop=True)   # Reset index.

In [13]:
def batch_embedding(start, stop):   # Function to create embeddings given index.
    
    global df   # Import df.
    
    # First observation.
    words = df.at[start,"full_text"].split(" ")   # Collect tokens of text.

    text_vec = np.array([0] * 50)   # Empty array.
    ttl_wrd = 0    # Empty count.

    for j in words:
        try:
            text_vec = text_vec + tr_wv[j]   # Vectorize and add.
            ttl_wrd += 1   # Increment count.
        except:    # Ignore if can't vectorize.
            pass

    x_0 = text_vec/ttl_wrd   # Compute mean.
    x_0 = x_0.tolist()    # Convert to list.

    words = df.at[start, "summary"].split(" ")    # Import summary.
    inp = [0] * 150   # Empty trigram.
    inp = x_0 + inp + [words[0]]   # mean vector, trigram, output word.
    full_inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))   # Create data frame of inputs.

    for j in range(1, len(words)):   # For each other word.
        try:
            inp = tr_wv[j-1].tolist()   # Convert last word to vector.
        except:
            inp = [0] * 50   # If can't vectorize, impute empty.
        # Mean vector, last word, two words prior to last, output word.
        inp = x_0 + inp + list(full_inp.iloc[j-1, 50:150]) + [words[j]]
        inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))   # Convert to df.
        full_inp = pd.concat([full_inp, inp]).reset_index(drop=True)   # Concatenate all.
    
    for i in range(start+1, stop):     # For every other file in range.
        # Repeat process.
        # Open tokens of text.
        words = df.at[i,"full_text"].split(" ")
        
        # Conmpute mean vector as before.
        text_vec = np.array([0] * 50)
        ttl_wrd = 0

        for j in words:
            try:
                text_vec = text_vec + tr_wv[j]
                ttl_wrd += 1
            except:
                pass

        x_0 = text_vec/ttl_wrd
        x_0 = x_0.tolist()

        # Read in summary tokens..
        words = df.at[i, "summary"].split(" ")
        # Create first input vector as before.
        inp = [0] * 150
        inp = x_0 + inp + [words[0]]
        # Sub-DF to hold sentences of this text.
        sub_inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))
        
        # Process every word after first as before.
        for j in range(1, len(words)):
            try:
                inp = tr_wv[words[j-1]].tolist()
            except:
                inp = [0] * 50
            inp = x_0 + inp + list(sub_inp.iloc[j-1, 50:150]) + [words[j]]
            inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))
            sub_inp = pd.concat([sub_inp, inp]).reset_index(drop=True)
            
        # Combine sub-DF to full DF.
        full_inp = pd.concat([full_inp, sub_inp]).reset_index(drop=True)
        
        # Print current process.
        clear_output(wait=True)
        print(f"{i+1}/{len(df.index)} Complete")
    
    # Save train df as pkl object.
    with open(f'/home/x-mbemus/Desktop/Checkpoints/NN_Base_train{stop}.pkl','wb') as f:
        pkl.dump(full_inp,f)

## Running Model!!!

In [17]:
# Embedd within each range.
ends = [[0, 50], [50, 100], [100, 150], [150, 200], [200, 250], [250, 300], 
        [300, 350], [350, 400], [400, 450], [450, 500], [500, 553]]

for end in ends:
    batch_embedding(end[0], end[1])

553/553 Complete


In [20]:
batch_embedding(0, 50)   # Testing function.

50/553 Complete


In [4]:
with open(f'/home/x-mbemus/Desktop/Checkpoints/NN_Base_train50.pkl','rb') as f:
    tr_data = pkl.load(f)   # Load first 50 input sets.

for i in [100, 150, 200, 250, 300, 350, 400, 450, 500, 553]:
    with open(f'/home/x-mbemus/Desktop/Checkpoints/NN_Base_train{i}.pkl','rb') as f:
        sub_data = pkl.load(f)   # Load each other input set.
        tr_data = pd.concat([tr_data, sub_data]).reset_index(drop=True)   # Concatenate consecutively.

In [5]:
# Assign d-type as before. 
d_dict = {}
for j in range(0, 200): 
    d_dict[j] = float 
    d_dict[200] = str

tr_data = tr_data.astype(d_dict)

In [6]:
# Create Classifier.
base_nn = MLPClassifier(random_state=42, hidden_layer_sizes=(100, 100, 100), max_iter=1000)

# Train classifier.
base_nn.fit(tr_data.iloc[:, 0:200], tr_data.iloc[:, 200])

# Save classifier.
with open(f'/home/x-mbemus/Desktop/Checkpoints/NN_Base_real.pkl','wb') as f: 
    pkl.dump(base_nn,f)

base_nn = MLPClassifier(random_state=42, hidden_layer_sizes=(100, 100, 100), max_iter=1000)

d_dict = {}
for j in range(0, 200):
    d_dict[j] = float
    d_dict[200] = str


words = df.at[0,"full_text"].split(" ")

text_vec = np.array([0] * 50)
ttl_wrd = 0

for j in words:
    try:
        text_vec = text_vec + tr_wv[j]
        ttl_wrd += 1
    except:
        pass

x_0 = text_vec/ttl_wrd
x_0 = x_0.tolist()

words = df.at[0, "summary"].split(" ")
inp = [0] * 150
inp = x_0 + inp + [words[0]]
full_inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))

for j in range(1, len(words)):
    try:
        inp = tr_wv[j-1].tolist()
    except:
        inp = [0] * 50
    inp = x_0 + inp + list(full_inp.iloc[j-1, 50:150]) + [words[j]]
    inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))
    full_inp = pd.concat([full_inp, inp]).reset_index(drop=True)
    
for i in range(1, len(df.index)):  
    #try:
        words = df.at[i,"full_text"].split(" ")

        text_vec = np.array([0] * 50)
        ttl_wrd = 0

        for j in words:
            try:
                text_vec = text_vec + tr_wv[j]
                ttl_wrd += 1
            except:
                pass

        x_0 = text_vec/ttl_wrd
        x_0 = x_0.tolist()

        words = df.at[i, "summary"].split(" ")
        inp = [0] * 150
        inp = x_0 + inp + [words[0]]
        sub_inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))

        for j in range(1, len(words)):
            try:
                inp = tr_wv[words[j-1]].tolist()
            except:
                inp = [0] * 50
            inp = x_0 + inp + list(sub_inp.iloc[j-1, 50:150]) + [words[j]]
            inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))
            sub_inp = pd.concat([sub_inp, inp]).reset_index(drop=True)
            
        full_inp = pd.concat([full_inp, sub_inp]).reset_index(drop=True)
        
        clear_output(wait=True)
        print(f"{i+1}/{len(df.index)} Complete")
        
        if i%100 == 0:
            with open(f'/home/x-mbemus/Desktop/Checkpoints/NN_Base_train{i}.pkl','wb') as f:
                pkl.dump(full_inp,f)
    #except:
    #    pass

with open(f'/home/x-mbemus/Desktop/Checkpoints/NN_Base_train_FULL.pkl','wb') as f:
    pkl.dump(full_inp,f)
    
full_inp.replace(["", np.nan], 0)

full_inp = full_inp.astype(d_dict) 

print("Training Model...")

base_nn.fit(full_inp.iloc[:, 0:200], full_inp.loc[:, 200])

with open(f'/home/x-mbemus/Desktop/Checkpoints/NN_Base_full.pkl','wb') as f:
            pkl.dump(base_nn,f)
        
print("Training Complete!!!")

In [4]:
# Open classifier.
with open('/home/x-mbemus/Desktop/Checkpoints/NN_Base_real.pkl', 'rb') as f:
    base_nn = pkl.load(f)

In [5]:
# Import metrics.
from sacrebleu.metrics import BLEU
bleu_scorer = BLEU(effective_order=True)

from rouge import Rouge
rouge_scorer = Rouge()

In [6]:
test_df = pd.read_csv("/home/x-mbemus/Desktop/text_test_data.csv")   # Open test df.

In [7]:
# Single test observations.
h = 1   # Observation 1.
# Read in file.
f = open("/home/x-mbemus/Desktop/text_files/" + test_df.at[h, "file_name"], "r", errors="ignore")

# Preprocess text.
text = ' '.join(f.readlines())
text = re.sub(r"\\n|\\t", " ", text)
text = re.sub(r"([^\w\s'])", r" \1 ", text)
text = re.sub(r"\s+", " ", text)
words = text.split(" ")

# Create mean vector.
text_vec = np.array([0] * 50)
ttl_wrd = 0

for j in words:
    try:
        text_vec = text_vec + tr_wv[j]
        ttl_wrd += 1
    except:
        pass

x_0 = text_vec/ttl_wrd
x_0 = x_0.tolist()

# Read in target summary.
target = test_df.at[h, "summary"]
# Empty vector.
inp = [0] * 100
inp = x_0 + tr_wv[''].tolist() + inp   # Been using "" as start and end token.
inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))   # Create df from first observation.
        
y_i = base_nn.predict(inp.iloc[:, 0:200])[0]   # Do first predition.
full_output = y_i   # String for full output of test..

itera = 1   # Iterable.
while y_i != '' and itera < 1034:   # If not stop token and less than max summary length.
    try:   # Convert last word to vector. 
        x_1 = tr_wv[y_i]
        x_1 = x_1.tolist()
        if len(x_1) == 1:   # Sometimes would create list with 1 observation.
            x_1 = x_1[0]   # Extract vector from outer list. 
    except:
        x_1 = [0] * 50   # Impute empty if conversion impossible.
    inp = x_0 + x_1 + list(inp.iloc[0, 50:150])   # Mean vec, last vec, two vecs prior.
    inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))   # Convert to df.
            
    y_i = base_nn.predict(inp.iloc[:, 0:200])[0]   # Predict with model.
    if y_i in [".", ",", "!", "?"]:   # If punctuation, just add.
        full_output = full_output + y_i
    else:
        full_output = full_output + " " + y_i   # Otherwise, add with space.
        
    itera += 1   # Increment iterable.
    # Check progress.
    clear_output(wait=True)
    print(f"{itera} Words Generated...")
        
# Remove extra spacing on ends.
full_output = full_output.strip()

# Compute scores.
blue = bleu_scorer.sentence_score(hypothesis=full_output, references=[target])
red = rouge_scorer.get_scores(hyps=full_output, refs=target)
        
# Print sample output.
print(full_output)
print(blue)
print(red)

1034 Words Generated...
The novel is a direct theme of the elves, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the emaciated, and the crew are greeted by the

In [13]:
# Dictionary of results.
results = {"title":[], "target":[], "pred":[], "BLEU":[], "ROUGE":[]}

exceptions = []   # List of exceptions.

# Repeat process in cell above for each test observation.
for h in test_df.index:
    try:
        f = open("/home/x-mbemus/Desktop/text_files/" + test_df.at[h, "file_name"], "r", errors="ignore")
    
        text = ' '.join(f.readlines())
        text = re.sub(r"\\n|\\t", " ", text)
        text = re.sub(r"([^\w\s'])", r" \1 ", text)
        text = re.sub(r"\s+", " ", text)
        words = text.split(" ")

        text_vec = np.array([0] * 50)
        ttl_wrd = 0

        for j in words:
            try:
                text_vec = text_vec + tr_wv[j]
                ttl_wrd += 1
            except:
                pass

        x_0 = text_vec/ttl_wrd
        x_0 = x_0.tolist()
        
        target = test_df.at[h, "summary"]
        
        inp = [0] * 100
        inp = x_0 + tr_wv[''].tolist() + inp
        inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))
        
        y_i = base_nn.predict(inp.iloc[:, 0:200])[0]
        full_output = y_i
        
        out_len = 1
        while y_i != '' and out_len < 1034:
            try:
                x_1 = tr_wv[y_i]
                x_1 = x_1.tolist()
                if len(x_1) == 1:
                    x_1 = x_1[0]
            except:
                x_1 = [0] * 50
            inp = x_0 + x_1 + list(inp.iloc[0, 50:150])
            inp = pd.DataFrame(np.array(inp).reshape(1, len(inp)))
            
            y_i = base_nn.predict(inp.iloc[:, 0:200])[0]
            if y_i in [".", ",", "!", "?"]:
                full_output = full_output + y_i
            else:
                full_output = full_output + " " + y_i
            
            out_len += 1
        
        full_output = full_output.strip()
        
        blue = bleu_scorer.sentence_score(hypothesis=full_output, references=[target])
        
        red = rouge_scorer.get_scores(hyps=full_output, refs=target)
        
        # Save results to result dictionary.
        results["title"].append(test_df.at[h, "title"])
        results["target"].append(target)
        results["pred"].append(full_output)
        results["BLEU"].append(blue.score)
        results["ROUGE"].append(red[0]["rouge-l"]["f"])
    
        test_df.at[h, "flagged"] = 0   # Do not flag.
        
    except Exception as e:   # If issues arose, flag.
        exceptions.append((h, e))
        test_df.at[h, "flagged"] = 1
    
    # Check progress.
    clear_output(wait=True)
    print(f'{h+1}/{len(test_df.index)}')

test_results = pd.DataFrame(results)   # Convert dict to df.
# Save df.
test_results.to_csv("/home/x-mbemus/Desktop/BASENN_test_result2.csv", index=False)

167/167


In [14]:
# Check results.
print(test_results['BLEU'].mean())
print(test_results['ROUGE'].mean())

0.38599607270217895
0.054923850539211366


In [15]:
print(test_results.at[0, "pred"])    # Test observation.

The novel is extremely unfavorable. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximately 500. The story is set approximate

In [6]:
# Find max summary length.
maximum = 0   # Min length.
for i in df.index:   # Iterate through df.
    text = df.at[0, "summary"]   # Extract summary.
    # Preprocess as before.
    text = re.sub(r"\\n|\\t", " ", text)
    text = re.sub(r"\\'", " ", text)
    text = re.sub(r"([^\w\s'])", r" \1 ", text)
    text = re.sub(r"\s+", " ", text)
    words = text.split(" ")   # Split by space.
    
    if len(words) > maximum:   # If longest summary so far.
        maximum = len(words)   # Assign as new max.
    
    # Track progress.
    clear_output(wait=True)
    print(f"{i+1}/{len(df.index)} Summaries Checked")
    print(f"Current Max: {maximum}")

666/666 Summaries Checked
Current Max: 1034
